# Programming

This notebook serves to capture the calculation and programing side of the investigation.



In [1]:
import os
import sys
import numpy as np
import pandas as pd
from datetime import date
from collections import OrderedDict
from dateutil.relativedelta import *
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize
from IPython.core.display import display, HTML

persfin = os.path.join("./","persfin")
sys.path = [persfin]+sys.path

import fingenerators as fingen


# pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('precision',5)
pd.set_option('display.max_colwidth',100)
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline
plt.style.use('ggplot')

## Mortgage Bond Amortisation Model

This model and code is taken from http://pbpython.com/amortization-model-revised.html. Moffit was looking for a way to do amortisation  calculations in Pandas without loops, but ended up using a generator function to do the looping.  The generator does a complete set of loops and then provides the answer to a Pandas DataFrame.   The power of Pandas is therefore not available during the loop, but it is available afterwards for analysis and plotting.  Moffit's original code is heavily refactored and cleaned up here. 

The algorithm to be executed at every time interval (e.g. every month) is as follows:

1. Calculate the interest on the balance, rounded to two decimals.
1. Calculate the payment as the smaller of the outstanding balance or the contractual payment amount (to account for a smaller payment in the last cycle).
1. Calculate the additional payment as the smaller of the outstanding balance or the fixed additional payment amount (to account for a smaller payment in the last cycle). 
1. Calculate the new balance by adding the interest and subtracting the two payments (but account for the signs of the financial values).
1. Calculate some housekeeping values like the date and payment cycle number.

It should be evident that the interest is calculated compound at each payment cycle, hence it adjusts at every payment cycle. 

The `amortise` function below is a Python generator function that provides iterative results for each time interval.  This generator is used to populate a Pandas DataFrame for subsequent anaysis.

The amortisation function is implemented as `fingenerators.amortise()` in the `persfin` module.

Confirm that the `amortise` function is a generator:

In [2]:
fingen.amortise(principal=10000, interest_rate=.03, bondyears=5, reqpayment=-400, addpayment=0, start_date=date(2000, 1,1), cyclesPerAnnum=12)

<generator object amortise at 0x000000000906DB88>

Next create a wrapper function to use `amortise` to fill a Pandas DataFrame.  The `amortisation_table` function creates and then cleans up the table and provides summary stats so it is easy to compare various scenarios.  See the function docstring for the parameters and conventions  used.


## Mortgage Bond Amortisation 

This model and code and code details are shown in the `00-Programming` notebook.

Imagine an account that keeps track of all money flowing in and out on a monthly basis.  The account is iterated on monthly until some end condition is met.  This end condition can be a fixed term or until the bond is repaid (balance is zero).

The main inputs to the model are the following:

1. `principal`: Amount borrowed
1. `interest_rate`: The annual interest rate for this loan.
1. `bondyears`: Bond duration in the number of years for the loan.
1. `reqpayment`: Required payment (minimum) amount per period to repay the loan during the term.
1. `addpayment`: Initial value of additional payments (if any) to be made each period.
1. `addpayrate`: Rate of increase in the  additional payment (can be zero), calculated once per year on January 1.
1. `start_date`: Start date for the loan. The start date is normally 2000-01-01.
1. `cyclesPerAnnum`: Number of payment cycles in a year, normally 12.

Note that the additional payment can be specified as a money value or as a fraction of the required payment. Complex value notation is used where the money value is the real component (e.g., -2300, negative value) and the fractional value is the imaginary component (e.g., .02j, positive fraction). If the (negative) real component (money value) is given the (positive) imaginary component (fraction value) is ignored. For a fractional value of 0.02j the additional payment is 0.02 * `reqpayment`.

The model provides two outputs: a monthly schedule that shows the values for every month as well as a summary of the main values.

The `df` DataFrame provides a monthly view of the account balance, interest, payments, etc.

The `stats` DataFrame provide a summary of the total contract, showing the key values, including the required payment, additional payment (if provided) and the total interest over the full term of the contract.  Note that the principal value is positive (inflow) and the payments and interest are negative (outflow).  


In [5]:
# to calculate  a typical bond repayment case with a non-escalating 2% additional payment
principal = 1000000
interest_rate = 0.09
bondyears = 20
cyclesPerAnnum = 12
reqpayment = round(np.pmt(interest_rate/cyclesPerAnnum, bondyears*cyclesPerAnnum, principal), 2)

df, stats = fingen.amortisation_table(
    principal=principal, 
    interest_rate=interest_rate, 
    bondyears=bondyears, 
    reqpayment = reqpayment,
    addpayment = 0.02j,
    cyclesPerAnnum=cyclesPerAnnum, 
    start_date=date(2000, 1,1),
    addpayrate=0.0,
    ID=""
    )


df['IntPercent'] = 100 * df['Interest'] / df['ReqPayment']
display(HTML(df.head().to_html()))
display(HTML(df.tail().to_html()))
display(HTML(stats.to_frame().head().to_html()))

,Period,Month,Begin Balance,ReqPayment,AddPayment,Interest,End Balance,Principal,InterestRate,ID,IntPercent
0,1,2000-01-01,1000000.0000,-8997.26,-179.9452,-7500.00,998322.7948,1000000,0.09,,83.35871
1,2,2000-02-01,998322.7948,-8997.26,-179.9452,-7487.42,996633.0096,1000000,0.09,,83.21889
2,3,2000-03-01,996633.0096,-8997.26,-179.9452,-7474.75,994930.5544,1000000,0.09,,83.07807
3,4,2000-04-01,994930.5544,-8997.26,-179.9452,-7461.98,993215.3292,1000000,0.09,,82.93614
4,5,2000-05-01,993215.3292,-8997.26,-179.9452,-7449.11,991487.2340,1000000,0.09,,82.79309


,Period,Month,Begin Balance,ReqPayment,AddPayment,Interest,End Balance,Principal,InterestRate,ID,IntPercent
223,224,2018-08-01,40116.8204,-8997.2600,-179.9452,-300.88,31240.4952,1000000,0.09,,3.34413
224,225,2018-09-01,31240.4952,-8997.2600,-179.9452,-234.30,22297.5900,1000000,0.09,,2.60413
225,226,2018-10-01,22297.5900,-8997.2600,-179.9452,-167.23,13287.6148,1000000,0.09,,1.85868
226,227,2018-11-01,13287.6148,-8997.2600,-179.9452,-99.66,4210.0696,1000000,0.09,,1.10767
227,228,2018-12-01,4210.0696,-4241.6496,-0.0000,-31.58,0.0000,1000000,0.09,,0.74452


,0
Principal,1000000
Payoff Date,2018-12-01 00:00:00
Num Payments,228
Interest Rate,0.09
BondYears,20


## Python and [module versions, and dates](https://github.com/rasbt/watermark)

In [6]:
# to get software versions
# https://github.com/rasbt/watermark
# https://github.com/rasbt/watermark/blob/master/docs/watermark.ipynb
# you only need to do this once
# pip install watermark
# conda install -c conda-forge watermark

%load_ext watermark
%watermark -v -m -p numpy,scipy -g 

CPython 3.7.1
IPython 7.2.0

numpy 1.15.4
scipy 1.1.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 7
machine    : AMD64
processor  : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
CPU cores  : 8
interpreter: 64bit
Git hash   : 08ad7046bb82c2bf082f0fd7a7215c8120cb7327
